In [239]:
import time
import datetime
import pandas as pd
import numpy as np
from collections import OrderedDict

#Analysis Data
#ETF: QYLD, NUSI, RYLD
#Stocks:AAPL, MS, MSFT
ticker = 'QYLD'
start_date = '2021-01-01'
end_date = '2021-10-25'

#Setup
target_dates = pd.date_range(start_date, end_date,freq='MS')#.strftime('%Y-%m-%d').tolist()
monthly_investment=100

period1 = int(time.mktime(datetime.datetime(2000, 1, 1, 23, 59).timetuple()))
period2 = int(time.mktime(datetime.datetime.now().timetuple()))
interval = '1d' # 1d, 1m

holdings_df = pd.DataFrame(columns=["Date","Share Price","Shares Purchased", "Dividend", "Dividend Shares", "Dividend Payout"])
dividends_df = pd.DataFrame(columns=["Date","Share Price","Shares Purchased", "Dividend", "Dividend Shares", "Dividend Payout"])
dividends = list()

def record_dividend(date, dividend):
    entry = {"Date": date,
             "Dividend" : dividend,
             "Dividend Shares": holdings_df[:date]['Shares Purchased'].sum(),
             "Dividend Payout" : (holdings_df[:date]['Shares Purchased'].sum())*dividend
            }
    return entry

#Get date from feed(s)
stock_query_string = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={period1}&period2={period2}&interval={interval}&events=history&includeAdjustedClose=true'
stock_df = pd.read_csv(stock_query_string)

div_query_string = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={period1}&period2={period2}&interval={interval}&events=div&includeAdjustedClose=true'
div_df = pd.read_csv(div_query_string, parse_dates=['Date'])
div_df.set_index('Date', inplace=True)
div_df.sort_index(inplace=True)


#Calculate Holdings for begining of month
bus_days = map(lambda x: x + pd.offsets.BDay(), target_dates)
bus_day_list = list(bus_days)
close_prices = []
for day in bus_day_list:
    carr = stock_df.loc[stock_df.Date==day.date().strftime('%Y-%m-%d')].Close.values
    if carr.size == 0:
        next_day = day - pd.offsets.BDay() - pd.offsets.BDay()
        carr = stock_df.loc[stock_df.Date==day.date().strftime('%Y-%m-%d')].Close.values

    if carr.size!=0:
        close = carr[0]
        
    close_prices.append((day.date().strftime('%Y-%m-%d'),close))
    entry = {'Date':day.date(), 'Share Price': close, 'Shares Purchased':monthly_investment/close}
    holdings_df = holdings_df.append(entry, ignore_index=True)
    
holdings_df['Date'] = pd.to_datetime(holdings_df['Date'])
holdings_df.set_index('Date', inplace=True)

#Calculate Dividends - if any - as they occour
divs_to_consider_df = div_df.loc[start_date:end_date]
dividends = [record_dividend(i, dividend) for i, dividend in divs_to_consider_df.itertuples()]
dividends_df = dividends_df.append(dividends, ignore_index = True)
dividends_df['Date'] = pd.to_datetime(dividends_df['Date'])
dividends_df.set_index('Date', inplace=True)

holdings = pd.concat([holdings_df,dividends_df])

total_shares = 0
for price in close_prices:
    monthly_shares = monthly_investment/price[1]
    total_shares = total_shares + monthly_shares
    
value = total_shares * close_prices[-1][1]
print("would spend ", len(close_prices)*monthly_investment, " over ", len(close_prices), "months")
print("and would have: ", total_shares, " shares, worth ", value, " at ", close_prices[-1][1])
print("Dividends erned: ",holdings['Dividend Payout'].sum())
holdings.replace(np.nan,'').sort_index()


would spend  1000  over  10 months
and would have:  44.29431393252903  shares, worth  961.6295111808913  at  21.709999
Dividends erned:  49.75539752039378


,Share Price,Shares Purchased,Dividend,Dividend Shares,Dividend Payout
Date,,,,,
2021-01-04,22.690001,4.407228,,,
2021-01-19,,,0.229,4.407228,1.009255
2021-02-02,23.299999,4.291846,,,
2021-02-22,,,0.233,8.699073,2.026884
2021-03-02,22.67,4.411116,,,
2021-03-22,,,0.224,13.110189,2.936682
2021-04-02,22.67,4.411116,,,
2021-04-19,,,0.228,17.521305,3.994858
2021-05-03,22.530001,4.438526,,,


In [201]:
#dividends = []
dividends_df
holdings_df
pd.concat([holdings_df,dividends_df], axis=0).sort_index()

,Share Price,Shares Purchased,Dividend,Dividend Shares,Dividend Payout
Date,,,,,
2021-01-04,27.420000,36.469730,NaN,NaN,NaN
2021-01-20,NaN,NaN,0.180,36.469730,6.564551
2021-02-02,27.830000,35.932447,NaN,NaN,NaN
2021-02-24,NaN,NaN,0.180,72.402177,13.032392
2021-03-02,27.360001,36.549706,NaN,NaN,NaN
2021-03-24,NaN,NaN,0.175,108.951883,19.066580
2021-04-02,27.360001,36.549706,NaN,NaN,NaN
2021-04-21,NaN,NaN,0.180,145.501590,26.190286
2021-05-03,27.480000,36.390102,NaN,NaN,NaN


In [147]:
#div_df[(div_df['Date'] > start_date) & (div_df['Date'] < end_date)]
d = div_df.loc[start_date:end_date]
[(i,(holdings_df[:i]['Shares Purchased'].sum()),(holdings_df[:i]['Shares Purchased'].sum())*x) for i, x in d.itertuples()]

[(Timestamp('2021-02-05 00:00:00'), 15.135333732271544, 3.1027434151156665),
 (Timestamp('2021-05-07 00:00:00'), 38.66488058698623, 8.50627372913697),
 (Timestamp('2021-08-06 00:00:00'), 60.67784819494166, 13.349126602887166)]

In [144]:
div_df.itertuples()
#for p in close_prices:
#    print(p[1])
#div_df

In [20]:
#div_df.sort_values(by="Date")
bus_day_list

[Timestamp('2021-01-04 00:00:00', freq='MS'),
 Timestamp('2021-02-02 00:00:00', freq='MS'),
 Timestamp('2021-03-02 00:00:00', freq='MS'),
 Timestamp('2021-04-02 00:00:00', freq='MS'),
 Timestamp('2021-05-03 00:00:00', freq='MS'),
 Timestamp('2021-06-02 00:00:00', freq='MS'),
 Timestamp('2021-07-02 00:00:00', freq='MS'),
 Timestamp('2021-08-02 00:00:00', freq='MS'),
 Timestamp('2021-09-02 00:00:00', freq='MS'),
 Timestamp('2021-10-04 00:00:00', freq='MS')]